In [ ]:
import pandas as pd

# Cargar los datos
df = pd.read_csv('../results/datos_normalizados.csv', sep=',')
df_cluster = pd.read_csv('../results/paises_agrupados_por_completitud.csv', sep=',')

# Combinar usando nombres distintos de columna
df = df.merge(df_cluster[['País', 'Cluster']], left_on='Área', right_on='País', how='left')

# Eliminar la columna duplicada 'País' si ya no la necesitas
df = df.drop(columns='País')

# Verificar resultado
df.info()

In [ ]:
df = df[df['Cluster'] == 0]

In [ ]:
# Asumimos que df tiene columnas: ['Área', 'Producto', 'Elemento', 'Año', 'Valor']

# --- 1️⃣ Obtener el conjunto de elementos por producto ---
elements_per_product = (
    df.groupby("Producto")["Elemento"]
    .unique()
    .apply(set)
    .to_dict()
)

# --- 2️⃣ Verificar si todos los productos tienen el mismo conjunto de elementos ---
all_elements = set.union(*elements_per_product.values())  # conjunto total
incomplete = {
    prod: all_elements - elems
    for prod, elems in elements_per_product.items()
    if elems != all_elements
}

print("Elementos totales encontrados:", all_elements)
print(f"Total de productos analizados: {len(elements_per_product)}")
print(f"Productos con elementos incompletos: {len(incomplete)}")

# --- 3️⃣ Mostrar detalle de productos incompletos ---
if incomplete:
    print("\n--- Productos con elementos faltantes ---")
    for prod, missing in incomplete.items():
        present = elements_per_product[prod]
        print(f"Producto: {prod}")
        print(f"  → Elementos presentes: {sorted(list(present))}")
        print(f"  → Elementos faltantes: {sorted(list(missing))}\n")

# --- 4️⃣ (opcional) Resumen tabular de completitud ---
summary = []
for prod, elems in elements_per_product.items():
    summary.append({
        "Producto": prod,
        "Elementos presentes": len(elems),
        "Elementos esperados": len(all_elements),
        "Porcentaje completo": round(100 * len(elems) / len(all_elements), 2)
    })
df_summary = pd.DataFrame(summary).sort_values("Porcentaje completo")



In [ ]:
print("\nResumen de completitud por producto:")
df_summary

In [ ]:
# --- 1️⃣ Verificar si hay duplicados ---
duplicates = df.duplicated(subset=["Área", "Producto", "Elemento", "Año"], keep=False)
if duplicates.any():
    print(f"⚠️ Se encontraron {duplicates.sum()} filas duplicadas. Se promediarán los valores repetidos.")
    # Agrupamos y promediamos los valores repetidos
    df = (
        df.groupby(["Área", "Producto", "Elemento", "Año"], as_index=False)
          .agg({"Valor": "mean"})
    )
else:
    print("✅ No hay duplicados exactos por combinación de claves.")

# --- 2️⃣ Crear el índice completo ---
all_areas = df["Área"].unique()
all_products = df["Producto"].unique()
all_elements = df["Elemento"].unique()
all_years = sorted(df["Año"].unique())

full_index = pd.MultiIndex.from_product(
    [all_areas, all_products, all_elements, all_years],
    names=["Área", "Producto", "Elemento", "Año"]
)

# --- 3️⃣ Reindexar el dataframe ---
df_full = (
    df.set_index(["Área", "Producto", "Elemento", "Año"])
      .reindex(full_index)
      .reset_index()
)

# --- 4️⃣ Detectar huecos ---
missing_mask = df_full["Valor"].isna()
missing = df_full[missing_mask]

print(f"🔍 Total de combinaciones posibles: {len(df_full):,}")
print(f"❌ Huecos encontrados: {missing_mask.sum():,}")
print(f"✅ Datos completos: {len(df_full) - missing_mask.sum():,}")

# --- 5️⃣ Mostrar resumen por país ---
resumen = (
    missing.groupby("Área")
    .size()
    .reset_index(name="Huecos")
    .sort_values("Huecos", ascending=False)
)
print("\n--- Países con datos incompletos ---")
print(resumen.head(10))

# --- 6️⃣ (Opcional) Exportar para inspección manual ---
missing.to_csv("../results/missing_combinations.csv", index=False)
print("\n💾 Archivo 'missing_combinations.csv' guardado con los huecos detectados.")


In [ ]:
map_1990 = {
    # --- categorías iguales ---
    'Residuos agrícolas': 'Residuos agrícolas',
    'Cultivo del arroz': 'Cultivo del arroz',
    'Quemado de residuos agrícolas': 'Quemado de residuos agrícolas',
    'Fermentación entérica': 'Fermentación entérica',
    'Gestión del estiércol': 'Gestión del estiércol',
    'Estiércol depositado en las pasturas': 'Estiércol depositado en las pasturas',
    'Estiércol aplicado a los suelos': 'Estiércol aplicado a los suelos',
    'Fertilizantes sintéticos': 'Fertilizantes sintéticos',
    'Energía': 'Energía',
    'IPPU': 'IPPU',
    'Desechos': 'Desechos',
    'Otro': 'Otro',
    'Emisiones derivadas de los cultivos': 'Emisiones derivadas de los cultivos',
    'Emissiones derivadas del sector ganadero': 'Emissiones derivadas del sector ganadero',
    'IPCC Agricultura': 'IPCC Agricultura',
    'Suelos agrícolas': 'Suelos agrícolas',

    # --- nuevas categorías agrupadas ---
    # Suelos, uso de la tierra, y drenaje
    'Suelos orgánicos drenados': 'Suelos agrícolas',
    'Suelos orgánicos drenados (CO2)': 'Suelos agrícolas',
    'Suelos orgánicos drenados (N2O)': 'Suelos agrícolas',
    'Tierras forestales': 'Suelos agrícolas',
    'Conversión neta de bosques': 'Suelos agrícolas',
    'Cambios de uso de la tierra': 'Suelos agrícolas',
    'Emisiones en tierras agrícolas': 'Suelos agrícolas',
    'LULUCF': 'Suelos agrícolas',
    'AFOLU': 'Suelos agrícolas',
    'Emisiones totales incluyendo LULUCF': 'Suelos agrícolas',
    'Emisiones totales excluyendo LULUCF': 'Suelos agrícolas',

    # Incendios
    'Incendios de sabana': 'Residuos agrícolas',
    'Incendios en suelos de turba': 'Residuos agrícolas',
    'Incendios forestales': 'Residuos agrícolas',
    'Incendios en los bosques tropicales húmedos': 'Residuos agrícolas',

    # Energía dentro de la finca
    'On-farm energy use': 'Energía',
    'Tanques de combustible internacional': 'Energía',

    # Fabricación y procesamiento industrial
    'Fabricación de fertilizantes': 'IPPU',
    'Fabricación de pesticidas': 'IPPU',
    'Envasado alimentario': 'IPPU',
    'Transformación\xa0de alimentos': 'IPPU',

    # Sistemas agroalimentarios y comercio
    'Sistemas agroalimentarios': 'Otro',
    'Farm gate': 'Otro',
    'Pre y\xa0post-producción': 'Otro',
    'Venta de alimentos': 'Otro',
    'Consumo\xa0de alimentos en los hogares': 'Otro',

    # Residuos de sistemas alimentarios
    'Eliminación de desechos de sistemas agroalimentarios': 'Desechos',
}

In [ ]:
df['Producto'] = df['Producto'].replace(map_1990)

In [ ]:
df["Producto"].unique()

In [ ]:
df = df[[
    "Área",
    "Producto",
    "Elemento",
    "Año",
    "Valor"
]]

In [ ]:
df = df.dropna(subset=["Año", "Valor"])


In [ ]:
# Mapear cada categoría a un índice
area_to_idx = {area: i for i, area in enumerate(df["Área"].unique())}
prod_to_idx = {prod: i for i, prod in enumerate(df["Producto"].unique())}
elem_to_idx = {elem: i for i, elem in enumerate(df["Elemento"].unique())}
year_to_idx = {year: i for i, year in enumerate(sorted(df["Año"].unique()))}

# Agregar las columnas indexadas al DataFrame
df["area_idx"] = df["Área"].map(area_to_idx)
df["prod_idx"] = df["Producto"].map(prod_to_idx)
df["elem_idx"] = df["Elemento"].map(elem_to_idx)
df["year_idx"] = df["Año"].map(year_to_idx)


In [ ]:
import torch
import numpy as np

num_areas = len(area_to_idx)
num_prods = len(prod_to_idx)
num_elems = len(elem_to_idx)
num_years = len(year_to_idx)

# Inicializamos el tensor con NaN (o ceros si preferís)
tensor = torch.full(
    (num_areas, num_prods, num_elems, num_years),
    float('nan')
)

# Rellenamos con los valores existentes
for _, row in df.iterrows():
    a, p, e, y = row["area_idx"], row["prod_idx"], row["elem_idx"], row["year_idx"]
    tensor[a, p, e, y] = row["Valor"]


In [ ]:
# Calculamos la media y desviación ignorando NaN de forma manual
mask = ~torch.isnan(tensor)
mean_val = torch.sum(tensor[mask]) / mask.sum()
std_val = torch.sqrt(torch.sum(((tensor[mask] - mean_val) ** 2)) / mask.sum())

# AGREGAMOS EPSILON PARA EVITAR DIVISIÓN POR CERO
epsilon = 1e-8
std_val = std_val + epsilon

print(f"Media: {mean_val.item():.4f}")
print(f"Desviación estándar: {std_val.item():.4f}")

tensor_norm = (tensor - mean_val) / std_val

# Verificar que no hay NaN o Inf después de normalizar
print(f"NaN después de normalización: {torch.isnan(tensor_norm).sum().item()}")
print(f"Inf después de normalización: {torch.isinf(tensor_norm).sum().item()}")


In [ ]:
def generar_secuencias(tensor, window=5):
    """
    Crea pares (input_seq, target) para entrenamiento temporal.
    Cada secuencia tiene longitud 'window' y el target es el siguiente año.
    """
    X, y = [], []
    num_years = tensor.shape[-1]
    for t in range(num_years - window):
        X.append(tensor[..., t:t+window])
        y.append(tensor[..., t+window])
    return torch.stack(X), torch.stack(y)

X, y = generar_secuencias(tensor_norm, window=5)
print("Shape de X:", X.shape)  # (n_samples, areas, productos, elementos, años_ventana)
print("Shape de y:", y.shape)


In [ ]:
import torch
import pandas as pd
import plotly.express as px

# Sumamos NaN sobre elementos
nan_per_year = torch.isnan(tensor).sum(dim=2)  # (áreas, productos, años)
num_areas, num_prods, num_years = nan_per_year.shape

# Convertimos a DataFrame largo
data = []
for a in range(num_areas):
    for p in range(num_prods):
        for y in range(num_years):
            data.append({
                "Área": a,
                "Producto": p,
                "Año": 1961 + y,  # ajusta si tu primer año no es 1961
                "NaN_count": int(nan_per_year[a,p,y])
            })
df_plot = pd.DataFrame(data)

# Creamos el heatmap interactivo
fig = px.density_heatmap(
    df_plot,
    x="Producto",
    y="Área",
    z="NaN_count",
    animation_frame="Año",  # slider automático por año
    color_continuous_scale="Reds",
    labels={"NaN_count": "Cantidad de NaN"}
)
fig.update_layout(height=600, width=900, title_text="Valores faltantes por Área y Producto")
fig.show()

